In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data
from functions.plot_angle_gradient import plot_angle_gradient

In [ ]:
# Check that the Stokes object file exists
stokes_filename = "generated/objects/stokes.pickle"
my_file = Path(stokes_filename)
if not my_file.is_file():
    print("Stokes object does not exists (", stokes_filename,"), please run file init_data, cailbration and normalization scripts first")

# Load the objects from the files
print("Loading Stokes data from file", stokes_filename)
stokes_list = pickle.load(open(stokes_filename, "rb"))

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
# Calculate first derivative
I.calc_first_derivative()

In [ ]:
I_d = np.gradient(I.data_n, I.wave_array.ravel(), axis=2)
I_dd = np.gradient(I_d, I.wave_array.ravel(), axis=2)

In [ ]:
# Calculate second derivative
I.calc_second_derivative()

In [ ]:
# Plot derived Intensity
fig = I.plot_frame_d(25)
fig = plot_data(I_d[:,:,25])

In [ ]:
# Plot derived Intensity
fig = I.plot_frame_dd(25)
fig = plot_data(I_dd[:,:,25])

In [ ]:
plt.plot(I.data_n[450,450,:])
plt.grid()

In [ ]:
plt.plot(I_d[450,450,:],color='black')
plt.plot(I.data_d[450,450,:],color='purple')
plt.grid()

In [ ]:
plt.plot(I_dd[450,450,:],color='black')
plt.plot(I.data_dd[450,450,:],color='red')
plt.grid()

In [ ]:
# Define function to sum data along the third axis (sum all wavelengths)
def sw(data):
    return np.sum(data, axis=2)

In [ ]:
# Calculate the vertical (longitudinal) component

#NOTE: these values are divided into _1 for the first spectral line and _2 for the second spectral line
lambda0  = [6301.51, 6302.50]  # in Angstroms
gbar     = [1.669, 2.487] # Lozitsky
C1 = [4.6686e-13 * l**2 * g for l, g in zip(lambda0, gbar)]
print(f"C1_1: {C1[0]}, C1_2: {C1[1]}")

f = 1 # filling factor, assumed to be 1

line_cuttoff = 55 # index at which we divide the data, between the spectral lines

In [ ]:
Bv = [-sw((V.data_n * I_d)[:, :, :line_cuttoff]) / (C1[0] * f * sw((I_d ** 2)[:, :, :line_cuttoff])),
       -sw((V.data_n * I_d)[:, :, line_cuttoff:]) / (C1[1] * f * sw((I_d ** 2)[:, :, line_cuttoff:]))]

In [ ]:
plot_data(Bv[0], f'B longitudinal, first line')
plot_data(Bv[1], f'B longitudinal, second line')

In [ ]:
# Esto es sólo para ver que no hay una diferencia significativa entre las dos mitades
plt.plot((V.data_n * I_d).mean(axis=(0,1)), color='blue')
plt.plot((V.data_n * I_d)[:, :, :line_cuttoff].mean(axis=(0,1)), color='orange')
plt.plot(np.arange(line_cuttoff, (V.data_n * I_d).shape[2]), (V.data_n * I_d)[:, :, line_cuttoff:].mean(axis=(0,1)), color='black')

In [ ]:
plt.plot((I_d ** 2).mean(axis=(0,1)), color='blue')
plt.plot((I_d ** 2)[:, :, :line_cuttoff].mean(axis=(0,1)), color='orange')
plt.plot(np.arange(line_cuttoff, (V.data_n * I_d).shape[2]), (I_d ** 2)[:, :, line_cuttoff:].mean(axis=(0,1)), color='black')

In [ ]:
Bv_all = -sw((V.data_n * I_d)) / (C1[0] * f * sw((I_d ** 2)))
plot_data(Bv_all, f'B longitudinal, all values together, C1[0]')

Bv_all = -sw((V.data_n * I_d)) / (C1[1] * f * sw((I_d ** 2)))
plot_data(Bv_all, f'B longitudinal, all values together, C1[1]')

In [ ]:
# Intento de sacar los arrays para ver si llego a algo xd
Vtest = V.data_n[:, :, :line_cuttoff]
Itest = I_d[:, :, :line_cuttoff]

Bv_test = -sw(Vtest*Itest) / (C1[0] * f * sw(Itest**2))
plot_data(Bv_test)

In [ ]:
# Calculate the horizontal (transverse) component

Gbar = [g**2 for g in gbar]  # Landi Degl'Innocenti & Landolfi (2004), si la línea es un triplete #TODO
C2 = [5.4490e-26 * l**4 * g for l, g in zip(lambda0, Gbar)]
print(f"C2_1: {C2[0]}, C2_2: {C2[1]}")

In [ ]:
L = np.sqrt(Q.data_n**2 + U.data_n**2)
Bt = [sw(L[:,:,:line_cuttoff] * np.abs(I_dd[:,:,:line_cuttoff])) / (C2[0] * f * sw(np.abs(I_dd[:,:,:line_cuttoff]))**2),
       sw(L[:,:,line_cuttoff:] * np.abs(I_dd[:,:,line_cuttoff:])) / (C2[1] * f * sw(np.abs(I_dd[:,:,line_cuttoff:]))**2)]
Bt = np.sqrt(Bt)

In [ ]:
plot_data(Bt[0], f'B transverse, first line')
plot_data(Bt[1], f'B transverse, second line')

In [ ]:
# Compute the azimuth angle (equal for both lines, does not depend on wavelength or Landé factor)
chi = np.arctan2(sw(U.data_n * I_dd), sw(Q.data_n * I_dd)) / 2

In [ ]:
plot_angle_gradient(chi, title='Azimuth angle')

In [ ]:
#TODO: make plot with lines projected over the image, like in Figure 5.2 of Jennerholm Hammar

In [ ]:
# Calculate inclination angle

const = [4/3 * g**2/G for g, G in (gbar, Gbar)]

num = [sw(np.abs(I.wave_array[:line_cuttoff] - lambda0[0]) * np.abs(L[:,:,:line_cuttoff]) * V.data_n[:,:,:line_cuttoff]**2 * np.abs(I.data_d[:,:,:line_cuttoff])),
         sw(np.abs(I.wave_array[line_cuttoff:] - lambda0[1]) * np.abs(L[:,:,line_cuttoff:]) * V.data_n[:,:,line_cuttoff:]**2 * np.abs(I_d[:,:,line_cuttoff:]))]
denom = [sw(np.abs(V.data_n[:,:,:line_cuttoff])**4),
         sw(np.abs(V.data_n[:,:,line_cuttoff:])**4)]
tan2gamma = [const[0] * num[0] / denom[0],
             const[1] * num[1] / denom[1]]
gamma = np.arctan(np.sqrt(tan2gamma))

In [ ]:
# Plot inclination for first line
plot_angle_gradient(gamma[0], title='Inclination, first line')

In [ ]:
# Plot inclination for second line
plot_angle_gradient(gamma[1], title='Inclination, second line')